In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
load_dotenv(override=True)

api_key = os.getenv("OPENAI_API_KEY")
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

# Check the API key
if (not api_key) and (not openrouter_api_key) and (not groq_api_key):
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif (not api_key.startswith("sk-proj-")) and (not openrouter_api_key.startswith("sk-")) and (not groq_api_key.startswith("gsk-")):
    print("An API key was found, but it is in the wrong format; please check you're using the right key - see troubleshooting notebook")
elif (api_key.strip() != api_key) and (openrouter_api_key.strip() != openrouter_api_key) and (groq_api_key.strip() != groq_api_key):
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [3]:
def shout(text):
    print(f"Shout has been called with input {text}")
    return text.upper()

In [4]:
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True, auth=[("student", "llmrocks"), ("teacher", "llmrocks")])

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Shout has been called with input Hello


In [5]:
message_input = gr.Textbox(label="Enter your message here", info="Type the message you want to send to the model", placeholder="Type your message...", lines=10)
message_output = gr.Textbox(label="Model Response", info="The response from the model will appear here", lines=10)

iface = gr.Interface(
    fn=shout,
    inputs=message_input,
    outputs=message_output,
    title="Message Shouter",
    description="A simple interface that shouts your message back at you in uppercase letters.",
    flagging_mode="never"
)
iface.launch(inbrowser=True, auth=[("student", "llmrocks"), ("teacher", "llmrocks")])

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Shout has been called with input Hello


In [6]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

In [7]:
# Let's wrap a call to llama 3.2 in a simple function

system_message = "You are a helpful assistant"

def message_gpt(prompt):
    messages = [{"role": "system", "content": system_message}, {"role": "user", "content": prompt}]
    response = ollama.chat.completions.create(model="llama3.2", messages=messages)
    return response.choices[0].message.content

In [ ]:
message_input = gr.Textbox(label="Enter your message here", info="Type the message you want to send to the model", placeholder="Type your message...", lines=10)
message_output = gr.Textbox(label="Model Response", info="The response from the model will appear here", lines=10)

iface = gr.Interface(
    fn=message_gpt,
    inputs=message_input,
    outputs=message_output,
    title="AI Chatbot",
    description="A simple call to AI Chatbot",
    flagging_mode="never"
)
iface.launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


In [12]:
system_message = "You are a helpful assistant that responds in markdown without code blocks"

message_input = gr.Textbox(label="Your message:", info="Enter a message for the chatbot", lines=7)
message_output = gr.Markdown(label="Response:")

iface = gr.Interface(
    fn=message_gpt,
    inputs=message_input,
    outputs=message_output,
    title="AI Chatbot",
    description="A simple call to AI Chatbot",
    flagging_mode="never",
    examples=[
        "Explain the Transformer architecture to a layperson",
        "Explain the Transformer architecture to an aspiring AI engineer",
    ],
)
iface.launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


In [13]:
def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = ollama.chat.completions.create(
        model='llama3.2',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [14]:
message_input = gr.Textbox(label="Your message:", info="Enter a message for the chatbot", lines=7)
message_output = gr.Markdown(label="Response:")

iface = gr.Interface(
    fn=stream_gpt,
    inputs=message_input,
    outputs=message_output,
    title="AI Chatbot",
    description="A simple call to AI Chatbot",
    flagging_mode="never",
    examples=[
        "Explain the Transformer architecture to a layperson",
        "Explain the Transformer architecture to an aspiring AI engineer",
    ],
)
iface.launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


In [15]:
openai = OpenAI()

In [16]:
def stream_gpt_real(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-5-nano',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [22]:
def stream_model(prompt, model):
    if model == "GPT":
        result = stream_gpt_real(prompt)
    elif model == "LLAMA":
        result = stream_gpt(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [23]:
message_input = gr.Textbox(label="Your message:", info="Enter a message for the LLM", lines=7)
model_selector = gr.Dropdown(["GPT", "LLAMA"], label="Select model", value="GPT")
message_output = gr.Markdown(label="Response:")

iface = gr.Interface(
    fn=stream_model,
    inputs=[message_input, model_selector],
    outputs=[message_output],
    title="AI Chatbot",
    flagging_mode="never",
    examples=[
        ["Explain the Transformer architecture to a layperson"],
        ["Explain the Transformer architecture to an aspiring AI engineer"]
    ],
)
iface.launch()

* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.
